In [371]:
%matplotlib inline

from functools import reduce
import datetime
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.preprocessing import MinMaxScaler

In [372]:
# Filenames
city_file = 'city_attributes.csv'
temp_file = 'temperature.csv'
humid_file = 'humidity.csv'
press_file = 'pressure.csv'
desc_file = 'weather_description.csv'
wdir_file = 'wind_direction.csv'
wspeed_file = 'wind_speed.csv'
# Load the files
city_df = pd.read_csv(city_file)
city_df.rename(str.lower, axis = 'columns', inplace = True)
city_df.drop(['country'], axis = 1, inplace = True)
city_df.set_index(['city'], inplace = True)
temp_df = pd.read_csv(temp_file)
humid_df = pd.read_csv(humid_file)
press_df = pd.read_csv(press_file)
desc_df = pd.read_csv(desc_file)
wdir_df = pd.read_csv(wdir_file)
wspeed_df = pd.read_csv(wspeed_file)

In [373]:
# These are the cities that universally have > 1% missing across all weather values
drop_city = set(temp_df.columns[temp_df.isna().sum() > 500]) & \
set(humid_df.columns[humid_df.isna().sum() > 500]) & \
set(press_df.columns[press_df.isna().sum() > 500]) & \
set(desc_df.columns[desc_df.isna().sum() > 500]) & \
set(wdir_df.columns[wdir_df.isna().sum() > 500]) & \
set(wspeed_df.columns[wspeed_df.isna().sum() > 500])  

In [374]:
# Remove the undesired cities and melt the tables to be conducive for joining
alt_temp_df = pd.melt(temp_df.drop(drop_city, axis = 1), id_vars = ['datetime'], var_name = 'city', value_name = 'temperature')
alt_humid_df = pd.melt(humid_df.drop(drop_city, axis = 1), id_vars = ['datetime'], var_name = 'city', value_name = 'humidity')
alt_press_df = pd.melt(press_df.drop(drop_city, axis = 1), id_vars = ['datetime'], var_name = 'city', value_name = 'pressure')
alt_desc_df = pd.melt(desc_df.drop(drop_city, axis = 1), id_vars = ['datetime'], var_name = 'city', value_name = 'weather_description')
alt_wdir_df = pd.melt(wdir_df.drop(drop_city, axis = 1), id_vars = ['datetime'], var_name = 'city', value_name = 'wind_direction')
alt_wspeed_df = pd.melt(wspeed_df.drop(drop_city, axis = 1), id_vars = ['datetime'], var_name = 'city', value_name = 'wind_speed')
# Set proper indices
alt_temp_df = alt_temp_df.set_index(['city', 'datetime'])
alt_humid_df = alt_humid_df.set_index(['city', 'datetime'])
alt_press_df = alt_press_df.set_index(['city', 'datetime'])
alt_desc_df = alt_desc_df.set_index(['city', 'datetime'])
alt_wdir_df = alt_wdir_df.set_index(['city', 'datetime'])
alt_wspeed_df = alt_wspeed_df.set_index(['city', 'datetime'])

In [375]:
# Join tables on the city and datetime info
dfs = [city_df, alt_temp_df, alt_humid_df, alt_press_df, alt_wspeed_df, alt_wdir_df, alt_desc_df]
df_final = reduce(lambda left, right : pd.merge(left, right, left_index = True, right_index = True), dfs)

In [376]:
# INTERPOLATION HAPPENS HERE -- Break up by city
df_final = df_final.groupby('city').apply(lambda group: group.interpolate(limit_direction = 'both'))

# Need to do something special for weather_description
arr, cat = df_final['weather_description'].factorize()
df_final['weather_description'] = pd.Series(arr).replace(-1, np.nan).\
interpolate(method = 'nearest', limit_direction = 'both')\
.interpolate(limit_direction = 'both').astype('category')\
.cat.rename_categories(cat).astype('str').values

In [377]:
# The whole purpose here is to encode wind direction. It's not continuous so don't really want to scale it
# Also have more granularity in wind dir if need be.
#dir_df = pd.DataFrame({'dir' : ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW', 'N'],
#                        'lower' : [348.75, 11.25, 33.75, 56.25, 78.75, 101.25, 123.75, 146.25, 168.75, 191.25, 213.75, 236.25, 258.75, 281.25, 303.75, 326.25, 0],
#                        'upper' : [360, 33.75, 56.25, 78.75, 101.25, 123.75, 146.25, 168.75, 191.25, 213.75, 236.25, 258.75, 281.25, 303.75, 326.25, 348.75, 11.25]})
dir_df = pd.DataFrame({'dir' : ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N'],
                        'lower' : [337.5, 22.5, 67.5, 112.5, 157, 202.5, 247.5, 292.5, 0],
                        'upper' : [360, 67.5, 112.5, 157, 202.5, 247.5, 292.5, 337.5, 22.5]})
# Make a copy to fool around in
fill_this = df_final['wind_direction'].copy()
# And overwrite the copy
for i in reversed(range(len(dir_df))):
#    print(str(dir_df.loc[i,'lower']) + " and " + str(dir_df.loc[i,'upper']))
    fill_this.loc[df_final['wind_direction'].between(dir_df.loc[i,'lower'], dir_df.loc[i,'upper'])] = i
# This is a bit ugly here; but it maintains any missing values nicely
df_final['wind_direction'] = dir_df.loc[fill_this, 'dir'].values

In [378]:
# Scaling happens here -- IMPUTATION MUST HAPPEN FIRST
scale_df = df_final[['latitude', 'longitude', 'temperature', 'humidity', 'pressure', 'wind_speed']].values
scaler = MinMaxScaler()
# We have access to min and max so we can transform back and forth
scale_df = scaler.fit_transform(scale_df)
print(scaler.data_min_)
print(scaler.data_max_)
df_final_scaled = df_final.copy()
df_final_scaled[['latitude', 'longitude', 'temperature', 'humidity', 'pressure', 'wind_speed']] = scale_df
df_final_scaled.head()

[  29.42412    -122.676208    242.33666667    5.          800.
    0.        ]
[  47.606209  -71.059769  321.22      100.       1100.         50.      ]


latitude  longitude  temperature  humidity  \
city     datetime                                                          
Portland 2012-10-01 12:00:00   0.88545        0.0     0.503824  0.800000   
         2012-10-01 13:00:00   0.88545        0.0     0.503824  0.800000   
         2012-10-01 14:00:00   0.88545        0.0     0.503865  0.789474   
         2012-10-01 15:00:00   0.88545        0.0     0.503975  0.789474   
         2012-10-01 16:00:00   0.88545        0.0     0.504084  0.789474   

                              pressure  wind_speed wind_direction  \
city     datetime                                                   
Portland 2012-10-01 12:00:00  0.746667         0.0              N   
         2012-10-01 13:00:00  0.746667         0.0              N   
         2012-10-01 14:00:00  0.746667         0.0              N   
         2012-10-01 15:00:00  0.746667         0.0              N   
         2012-10-01 16:00:00  0.746667         0.0             NE   

                             weather_description  
city     datetime                                 
Portland 2012-10-01 12:00:00    scattered clouds  
         2012-10-01 13:00:00    scattered clouds  
         2012-10-01 14:00:00    scattered clouds  
         2012-10-01 15:00:00    scattered clouds  
         2012-10-01 16:00:00    scattered clouds

In [379]:
# Collapse a lot of these groupings
weather_dict = {'scattered clouds' : 'partly_cloudy', 'sky is clear' : 'clear', 
             'few clouds' : 'partly_cloudy', 'broken clouds' : 'partly_cloudy',
           'overcast clouds' : 'cloudy', 'mist' : 'cloudy', 'haze' : 'cloudy', 
             'dust' : 'other', 'fog' : 'cloudy', 'moderate rain' : 'rain',
           'light rain' : 'rain', 'heavy intensity rain' : 'rain', 'light intensity drizzle' : 'rain',
           'heavy snow' : 'snow', 'snow' : 'snow', 'light snow' : 'snow', 'very heavy rain' : 'rain',
           'thunderstorm' : 'tstorm', 'proximity thunderstorm' : 'tstorm', 'smoke' : 'other', 'freezing rain' : 'snow',
           'thunderstorm with light rain' : 'tstorm', 'drizzle' : 'rain', 'sleet' : 'snow',
           'thunderstorm with rain' : 'tstorm', 'thunderstorm with heavy rain' : 'tstorm',
           'squalls' : 'rain', 'heavy intensity drizzle' : 'rain', 'light shower snow' : 'snow',
           'light intensity shower rain' : 'rain', 'shower rain' : 'rain',
           'heavy intensity shower rain' : 'rain', 'proximity shower rain' : 'rain',
           'proximity sand/dust whirls' : 'other', 'proximity moderate rain' : 'rain', 'sand' : 'other',
           'shower snow' : 'snow', 'proximity thunderstorm with rain' : 'tstorm',
           'sand/dust whirls' : 'other', 'proximity thunderstorm with drizzle' : 'tstorm',
           'thunderstorm with drizzle' : 'tstorm', 'thunderstorm with light drizzle' : 'tstorm',
           'light rain and snow' : 'snow', 'thunderstorm with heavy drizzle' : 'tstorm',
           'ragged thunderstorm' : 'tstorm', 'tornado' : 'other', 'volcanic ash' : 'other', 'shower drizzle' : 'rain',
           'heavy shower snow' : 'snow', 'light intensity drizzle rain' : 'rain',
           'light shower sleet' : 'snow', 'rain and snow' : 'snow'}

In [380]:
adj_weather = [weather_dict[val] for val in df_final_scaled['weather_description']]
df_final_scaled['adj_weather'] = adj_weather
df_final_scaled = df_final_scaled.drop('weather_description', axis = 1)

In [381]:
# And one-hot encode the wind_directions and weather_description
df_final_scaled = pd.get_dummies(df_final_scaled, prefix=['wind_dir', 'weather'], columns=['wind_direction', 'adj_weather'])

In [382]:
df_final_scaled = df_final_scaled.reset_index('city')
# Train, test, valid split
train_df = df_final_scaled[df_final_scaled.index < '2016-01-01 00:00:00']
test_df = df_final_scaled[df_final_scaled.index >= '2016-01-01 00:00:00']
test_df = test_df[test_df.index < '2017-01-01 00:00:00']
valid_df = df_final_scaled[df_final_scaled.index >= '2017-01-01 00:00:00']

In [385]:
train_df

,city,latitude,longitude,temperature,humidity,pressure,wind_speed,wind_dir_E,wind_dir_N,wind_dir_NE,...,wind_dir_SE,wind_dir_SW,wind_dir_W,weather_clear,weather_cloudy,weather_other,weather_partly_cloudy,weather_rain,weather_snow,weather_tstorm
datetime,,,,,,,,,,,,,,,,,,,,,
2012-10-01 12:00:00,Portland,0.885450,0.0,0.503824,0.800000,0.746667,0.00,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 13:00:00,Portland,0.885450,0.0,0.503824,0.800000,0.746667,0.00,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 14:00:00,Portland,0.885450,0.0,0.503865,0.789474,0.746667,0.00,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 15:00:00,Portland,0.885450,0.0,0.503975,0.789474,0.746667,0.00,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 16:00:00,Portland,0.885450,0.0,0.504084,0.789474,0.746667,0.00,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 17:00:00,Portland,0.885450,0.0,0.504193,0.778947,0.746667,0.00,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 18:00:00,Portland,0.885450,0.0,0.504302,0.778947,0.746667,0.00,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 19:00:00,Portland,0.885450,0.0,0.504411,0.768421,0.746667,0.00,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2012-10-01 20:00:00,Portland,0.885450,0.0,0.504521,0.768421,0.746667,0.00,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [386]:
# Write for distribution
#df_final_scaled.to_csv('~/Downloads/df_weather_scaled_encoded')

In [334]:
practice = test_df[test_df['city'] == 'Philadelphia']

In [339]:
practice.shape

(8784, 75)

In [338]:
seq_len = 24



,city,latitude,longitude,temperature,humidity,pressure,wind_speed,wind_dir_E,wind_dir_ENE,wind_dir_ESE,...,weather_thunderstorm,weather_thunderstorm with drizzle,weather_thunderstorm with heavy drizzle,weather_thunderstorm with heavy rain,weather_thunderstorm with light drizzle,weather_thunderstorm with light rain,weather_thunderstorm with rain,weather_tornado,weather_very heavy rain,weather_volcanic ash
datetime,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,Philadelphia,0.579043,0.92049,0.456666,0.810526,0.733333,0.06,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 01:00:00,Philadelphia,0.579043,0.92049,0.456429,0.810526,0.733333,0.06,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 02:00:00,Philadelphia,0.579043,0.92049,0.432100,0.810526,0.756667,0.04,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 03:00:00,Philadelphia,0.579043,0.92049,0.450074,0.810526,0.733333,0.04,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 04:00:00,Philadelphia,0.579043,0.92049,0.441454,0.800000,0.733333,0.04,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 05:00:00,Philadelphia,0.579043,0.92049,0.440772,0.800000,0.733333,0.04,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 06:00:00,Philadelphia,0.579043,0.92049,0.436003,0.578947,0.733333,0.08,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 07:00:00,Philadelphia,0.579043,0.92049,0.432960,0.673684,0.730000,0.02,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016-01-01 08:00:00,Philadelphia,0.579043,0.92049,0.429442,0.673684,0.733333,0.02,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# NEED TO
#   Perform imputation on missing values -- Probably by city and day -- DONE
#   Join the tables -- DONE
#   Do min-max scaling -- DONE
#   Roll up the values to the daily level -- NOT DOING (this isn't what we were planning on doing in our proposal)
#   Encode the weather_description and wind direction as a one-hot -- DONE
#   Get the wind direction as a categorical -- DONE

# Pretty good. Have some more to do now
#   Separate into training, testing, and validation --DONE
#   Fully break up the data into the Xtrain, Xtest, Xvalid, Ytrain, Ytest, and Yvalid